# **Cài đặt môi trường**

In [2]:
!pip install -q bitsandbytes datasets accelerate loralib evaluate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.6 MB/s eta 0:00:00
    

In [1]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
# Load model bloomz-1b1
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b1").to("cuda")
# Load tokenizer từ bloomz-1b1
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")

# **Tải dataset hỏi đáp web bán hàng**

In [3]:
!wget https://github.com/phatjkk/data/raw/main/LLM/Ecommerce_FAQ_Chatbot_dataset_vi.xlsx

--2023-07-15 08:41:41--  https://github.com/phatjkk/data/raw/main/LLM/Ecommerce_FAQ_Chatbot_dataset_vi.xlsx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/phatjkk/data/main/LLM/Ecommerce_FAQ_Chatbot_dataset_vi.xlsx [following]
--2023-07-15 08:41:42--  https://raw.githubusercontent.com/phatjkk/data/main/LLM/Ecommerce_FAQ_Chatbot_dataset_vi.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12116 (12K) [application/octet-stream]
Saving to: ‘Ecommerce_FAQ_Chatbot_dataset_vi.xlsx’

Ecommerce_FAQ_Chatb 100%[===================>]  11.83K  --.-KB/s    in 0s      

2023-07-15 08:41:42 (79.1 MB/s)

In [4]:
import pandas as pd
df = pd.read_excel("Ecommerce_FAQ_Chatbot_dataset_vi.xlsx", header=None,skiprows=1).drop(0)

In [5]:
df

,0,1
1,Làm thế nào tôi có thể tạo một tài khoản?,"Để tạo tài khoản, hãy nhấp vào nút 'Đăng ký' ..."
2,Phương thức thanh tóan nào được chấp nhận?,"Chúng tôi chấp nhận các thẻ tín dụng chính, t..."
3,Làm thế nào tôi có thể theo dõi đơn hàng của ...,Bạn có thể theo dõi đơn đặt hàng của mình bằn...
4,Chính sách hoàn trả của bạn là gì?,Chính sách hoàn trả của chúng tôi cho phép bạ...
5,Tôi có thể hủy đơn hàng của mình không?,Bạn có thể hủy đơn đặt hàng nếu nó chưa được v...
...,...,...
75,Tôi có thể đặt hàng nếu sản phẩm được liệt kê...,"Nếu một sản phẩm được liệt kê là ""hết hàng"" n..."
76,Tôi có thể trả lại sản phẩm nếu nó được mua b...,"Có, bạn có thể trả lại sản phẩm đã mua bằng t..."
77,Tôi có thể yêu cầu một sản phẩm nếu nó hiện kh...,Nếu một sản phẩm không có sẵn màu sắc ưa thíc...
78,Tôi có thể đặt hàng nếu sản phẩm đó được liệt...,"Nếu một sản phẩm được liệt kê là ""sắp ra mắt""..."


In [6]:
from datasets import load_dataset
from datasets import Dataset
Dataset.from_pandas(df)[0]

{'0': ' Làm thế nào tôi có thể tạo một tài khoản?',
 '1': " Để tạo tài khoản, hãy nhấp vào nút 'Đăng ký' ở góc trên cùng bên phải của trang web của chúng tôi và làm theo hướng dẫn để hoàn tất quy trình đăng ký."}

In [7]:
ds = Dataset.from_pandas(df)

In [8]:
# Kết hợp câu hỏi và câu trả lời liền kề nhau để model học nội dung
# Mục đích là để khi đưa 1 đoạn câu hỏi, model sẽ tự nhiểu và gen ra câu trả lời
# phía sau
def merge_columns(example):
    example["prediction"] = example["0"] + " ->: " + str(example["1"])
    return example

In [9]:
# Map function trên vào dataset
ds = ds.map(merge_columns)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [10]:
# Tokenizer/Véc tơ hóa văn bản (Chuyển văn bản thành số để training)
def tokeni(example):
    example["prediction_token"] = tokenizer(example["prediction"], return_tensors='pt', padding=True)['input_ids']
    return example
# Map function trên vào dataset
ds = ds.map(tokeni,batched=True)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [11]:
# Hàm tính số lượng parameters có thể train được trên model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
# Tạo LoraConfig để áp dụng Lora vào việc training mô hình
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, #attention heads
    lora_alpha=16, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1179648 || all params: 1066493952 || trainable%: 0.11060990995662018


In [13]:
# Training model bằng hàm trainer
import transformers
trainer = transformers.Trainer(
    model=model,
    train_dataset=ds["prediction_token"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        max_steps=150,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
        # remove_unused_columns=False
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,4.486900
2,4.400300
3,4.103400
4,4.650800
5,4.944100
6,3.275900
7,4.129800
8,4.186400
9,3.220900
10,4.499900


TrainOutput(global_step=150, training_loss=2.0299313565095267, metrics={'train_runtime': 25.3295, 'train_samples_per_second': 11.844, 'train_steps_per_second': 5.922, 'total_flos': 101958824411136.0, 'train_loss': 2.0299313565095267, 'epoch': 3.75})

In [14]:
tokenizer

BloomTokenizerFast(name_or_path='bigscience/bloomz-1b1', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [15]:
print(ds["0"][0])
print(ds["1"][0])

 Làm thế nào tôi có thể tạo một tài khoản?
 Để tạo tài khoản, hãy nhấp vào nút 'Đăng ký' ở góc trên cùng bên phải của trang web của chúng tôi và làm theo hướng dẫn để hoàn tất quy trình đăng ký.


In [16]:
question = str(ds["0"][0]+" ->: ")
question

' Làm thế nào tôi có thể tạo một tài khoản? ->: '

In [17]:
# In ra kết quả
inputs = tokenizer( question, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast(device.type):
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), max_new_tokens=100)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

 Làm thế nào tôi có thể tạo một tài khoản? ->:  Nhấp vào nút "Đăng ký" ở đầu trang để tạo tài khoản. Bạn có thể đăng ký bằng cách:  Đăng ký bằng tài khoản của mình. Nhập thông tin đăng ký của bạn và nhấp vào nút "Đăng ký". Nhấp vào nút "Đăng ký" ở đầu trang để tạo tài khoản. Bạn có thể đăng ký bằng cách:  Đăng ký bằng tài khoản của mình. Nhập thông tin đăng ký của bạn và nhấp vào nút "Đ


In [18]:
question="Làm sao để mua hàng?"
inputs = tokenizer( question, return_tensors="pt")
with torch.autocast(device.type):
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), max_new_tokens=100)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Làm sao để mua hàng? ->:  Chúng tôi cung cấp các tùy chọn mua hàng để bạn có thể mua hàng với nhiều lựa chọn. Chúng tôi sẽ liên hệ với bạn trong vài giờ để xác nhận đơn hàng của bạn. Chúng tôi sẽ cung cấp các tùy chọn thanh toán để bạn có thể thanh toán đơn hàng của mình. Chúng tôi sẽ liên hệ với bạn trong vài giờ để xác nhận đơn hàng của bạn.


# **Gen All question**

In [ ]:
gen_text = []
for i in range(len(ds["0"])):
  question=str(ds["0"][i]+" ->: ")
  inputs = tokenizer( question, return_tensors="pt")
  with torch.autocast("cuda"):
      outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
      tl = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
      gen_text.append(tl)
      print(tl)

 Làm thế nào tôi có thể tạo một tài khoản? ->:  Nhấp vào nút "Đăng ký" ở góc trên bên phải màn hình để tạo tài khoản. Bạn có thể đăng ký bằng cách cung cấp thông tin cá nhân và xác nhận bằng cách nhấp vào nút "Đăng ký" ở góc dưới bên phải màn hình. Sau khi tạo tài khoản, bạn có thể bắt đầu sử dụng tài khoản của mình. Nhấp vào biểu tượng tài khoản ở góc trên bên phải màn hình để bắt đầu sử dụng tài khoản của mình.
 Phương thức thanh tóan nào được chấp nhận? ->:  Có nhiều phương thức thanh toán được chấp nhận để thanh toán cho một sản phẩm. Vui lòng tham khảo các phương thức thanh toán được chấp nhận để xác định phương thức thanh toán phù hợp với bạn. Có nhiều phương thức thanh toán được chấp nhận để thanh toán cho một sản phẩm. Vui lòng tham khảo các phương thức thanh toán được chấp nhận để xác định phương thức thanh toán phù hợp với bạn.
 Làm thế nào tôi có thể theo dõi đơn hàng của mình? ->:  Nhấp vào nút "Đặt hàng" ở góc trên bên phải của trang web của chúng tôi để bắt đầu theo dõi đ